In [ ]:
import sys
import math
import boto3
import sagemaker
from time import gmtime, strftime

In [6]:
import sys
import math
import boto3
import sagemaker
from time import gmtime, strftime

role = sagemaker.get_execution_role()
region = boto3.Session().region_name
runtime_client = boto3.client("runtime.sagemaker", region_name=region)
client = boto3.client("sagemaker", region_name=region)

In [8]:
# Creating model endpoint
print(endpoint_name)
create_endpoint_response = client.create_endpoint(
    EndpointName=endpoint_name, EndpointConfigName=endpoint_config_name
)
print(create_endpoint_response["EndpointArn"])

resp = client.describe_endpoint(EndpointName=endpoint_name)
status = resp["EndpointStatus"]
while status == "Creating":
    print(f"Status: {status}")
    time.sleep(60)
    resp = client.describe_endpoint(EndpointName=endpoint_name)
    status = resp["EndpointStatus"]

print(f"Arn: {resp['EndpointArn']}")
print(f"Status: {status}")

ds-mlops-xgboost-endpoint
arn:aws:sagemaker:us-east-1:525102048888:endpoint/ds-mlops-xgboost-endpoint
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Arn: arn:aws:sagemaker:us-east-1:525102048888:endpoint/ds-mlops-xgboost-endpoint
Status: InService


## 7. Validate scoring

In this section we will validate scoring logics. In below code snippet we are scoring with 14 predictor values and finding the result

In [1]:
import boto3
import json

client = boto3.client('sagemaker-runtime')
test_vector = [2018,5,147,2,4,2,2000,0,2,3,8,6]
body={
  "features": {
"values": [2018,5,147,2,4,2,2000,0,2,3,8,6]
  }
}
#body = ','.join([str(item) for item in test_vector])
response_model_linear_learner = client.invoke_endpoint(EndpointName='wipcoe-pricing-ml-training-pipeline-lr-endpoint',
                               ContentType='application/json',
                               Body=body)

result_linear_learner = json.loads(str(response_model_linear_learner["Body"].read().decode('utf8')))
result_linear_learner = float(result_linear_learner["predictions"][0]["score"])
print(result_linear_learner)

ParamValidationError: Parameter validation failed:
Invalid type for parameter Body, value: {'features': {'values': [2018, 5, 147, 2, 4, 2, 2000, 0, 2, 3, 8, 6]}}, type: <class 'dict'>, valid types: <class 'bytes'>, <class 'bytearray'>, file-like object

In [24]:
payload={'year': 2018, 'manufacturer': 5, 'model': '147', 'condition': '2', 'cylinders': '4', 'fuel': '2', 'odometer': '2000', 'title_status': 0, 'transmission': '2', 'drive': '3', 'type': '8', 'paint_color': '6'}
scoring_vector = [payload['year'],payload['manufacturer'],payload['model'],payload['condition'],payload['cylinders'],payload['fuel'],payload['odometer'],payload['title_status'],payload['transmission'],payload['drive'],payload['type'],payload['paint_color']]
lrbodyvector = ([str(item) for item in scoring_vector])
lrbody=json.dumps({
    "instances":[
    {"features":lrbodyvector 
    }]})

client = boto3.client('sagemaker-runtime')
#test_vector = [2018,5,147,2,4,2,2000,0,2,3,8,6]
#body = ','.join([str(item) for item in test_vector])
response_model_linear_learner = client.invoke_endpoint(EndpointName='wipcoe-pricing-ml-training-pipeline-lr-endpoint',
                               ContentType='application/json',
                               Body=lrbody)
result_linear_learner = json.loads(str(response_model_linear_learner["Body"].read().decode('utf8')))
result_linear_learner = float(result_linear_learner["predictions"][0]["score"])
result_linear_learner=int(result_linear_learner)
print(result_linear_learner)

8314


In [25]:
payload={'year': 2018, 'manufacturer': 5, 'model': '147', 'condition': '2', 'cylinders': '4', 'fuel': '2', 'odometer': '2000', 'title_status': 0, 'transmission': '2', 'drive': '3', 'type': '8', 'paint_color': '6'}
scoring_vector = [payload['year'],payload['manufacturer'],payload['model'],payload['condition'],payload['cylinders'],payload['fuel'],payload['odometer'],payload['title_status'],payload['transmission'],payload['drive'],payload['type'],payload['paint_color']]
xgbody = ','.join([str(item) for item in scoring_vector])
client = boto3.client('sagemaker-runtime')
response_model_xgboost = client.invoke_endpoint(EndpointName='wipcoe-pricing-ml-training-pipeline-xgboost-endpoint',
                               ContentType='text/csv',
                               Body=xgbody)
result_xgboost = float(response_model_xgboost["Body"].read())
result_xgboost=int(result_xgboost)
print(result_xgboost)

8060


In [72]:
payload={'year': 2018, 'manufacturer': 5, 'model': '147', 'condition': '2', 'cylinders': '4', 'fuel': '2', 'odometer': '2000', 'title_status': 0, 'transmission': '2', 'drive': '3', 'type': '8', 'paint_color': '6'}
scoring_vector = [payload['year'],payload['manufacturer'],payload['model'],payload['condition'],payload['cylinders'],payload['fuel'],payload['odometer'],payload['title_status'],payload['transmission'],payload['drive'],payload['type'],payload['paint_color']]
body1 = ([str(item) for item in scoring_vector])
body=json.dumps({
    "instances":[
    {"features":body1 
    }]})
body2=json.dumps({
    "instances":[
    {"features":[2018,5,147,2,4,2,2000,0,2,3,8,6]
    }]})
print(body)
print(body2)
print(type(body1),type(body2))

{"instances": [{"features": ["2018", "5", "147", "2", "4", "2", "2000", "0", "2", "3", "8", "6"]}]}
{"instances": [{"features": [2018, 5, 147, 2, 4, 2, 2000, 0, 2, 3, 8, 6]}]}
<class 'list'> <class 'str'>


**Note:** Similarly you can change the test vector and can see how endpoint is working